# Load and predict multiple

This notebook loads multiple Keras Convolutional Neural Nets and combines their predictive powers.

In [1]:
from keras.models import load_model
from skimage.transform import resize
import csv
import h5py
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
%matplotlib inline

Using TensorFlow backend.


## Set global variables and model hyper-parameters

In [2]:
DATA_LOCATION = '../data/'
TEST_IMAGES_LOCATION = '../data/test_images/'
IMAGE_SIZE = 64
N_CLASSES = 121

filenames = [i for i in os.listdir('../data/test_images') if i.endswith('.jpg')]

## Helper functions

In [3]:
def get_padding(i):
    """
    Helper function for getting right padding sizes
    input:
        - i: positive integer gotten from substracting height and width of an image
    output:
        - Tuple representing the correct padding
    """
    if i%2 == 0:
        return (int(i/2),int(i/2))
    else:
        return (int(i/2-.5), int(i/2+.5))

def pad_image(img):
    """
    Add padding to image to make it square
    input:
        - img: numpy array (2D) representing image
    output:
        - padded array of shape (N,N)
    """
    H, W = img.shape
    if H == W:
        return img
    elif H > W:
        return np.pad(img, ((0,0), get_padding(H-W)), 'constant')
    else:
        return np.pad(img, (get_padding(W-H), (0,0)), 'constant')

def resize_image(img):
    """
    Resize image to new square shape
    input:
        - img: numpy array (2D) representing image
        - size: final shape of image in pixels (integer)
    """
    return resize(img, (IMAGE_SIZE,IMAGE_SIZE), mode='reflect')

## Load multiple models


In [4]:
model_3 = load_model('../data/output/models/model3.h5')
model_4 = load_model('../data/output/models/model4.h5')
model_5 = load_model('../data/output/models/model5.h5')
model_7 = load_model('../data/output/models/model7.h5')

models = [model_3,model_4,model_5,model_7]

In [5]:
def load_image(index, filename):
    # Read and transform image to usable format
    img = mpimg.imread(TEST_IMAGES_LOCATION + filename)
    img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
    img = resize_image(pad_image(img))
    # Create a grayscale channel 
    img = img.reshape(IMAGE_SIZE,IMAGE_SIZE,1)
    return img
    
# Do make sure all the models have the same input_shape
def make_predictions(models, filenames):
    X = np.empty([len(filenames),IMAGE_SIZE,IMAGE_SIZE,1])
    for i in range(len(filenames)):
        X[i] = load_image(i, filenames[i])
    
    predictions = {filename:np.nan for filename in filenames}
    for key in predictions.keys():
        predictions[key] = {i:0 for i in range(121)}
    
    # For each model ...
    for i in range(len(models)):
        print('Starting with model {} of {}'.format(i+1, len(models)))
        # Predict each test-image ...
        for j in range(X.shape[0]):
            filename = filenames[j]
            class_predictions = models[i].predict(X[j].reshape(1,IMAGE_SIZE,IMAGE_SIZE,1))[0]
            # For each class likelyhood ...
            for k in range(121):
                predictions[filename][k] += class_predictions[k]
    
    # Per image only take class with highest prediction-score
    for filename in filenames:
        predictions[filename] = max(predictions[filename], key=predictions[filename].get)
    
    return predictions

def print_model_summaries(models):
    for model in models:
        model.summary()

## Get the most predicted class per image

In [6]:
predictions = make_predictions(models, filenames)

In [17]:
labels = pd.DataFrame(list(predictions.items()), columns=['image', 'class'])
labels.sample(n=5)

,image,class
5334,81261.jpg,26
3128,29695.jpg,21
2418,157697.jpg,105
2149,150784.jpg,58
5739,91232.jpg,58


In [18]:
labels.to_csv('../data/output/predictions/model_combined_3_4_5.csv', index=False)